In [1]:
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
UNI = "cld2167"

In [2]:
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip().split("-")[1] for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0

    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [4]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train_total, y_train_total = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
# X_train=X_train_total[:,:45000]
# X_val=X_train_total[:,45000:]
# y_train=y_train_total[:45000]
# y_val=y_train_total[45000:]

## probably best to train on EVERYTHING


X_train=X_train_total[:,:45000]
X_val=X_train_total[:,45000:]
y_train=y_train_total[:45000]
y_val=y_train_total[45000:]

In [6]:
def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot
print(y_train.shape)
y_train=one_hot(y_train)
y_val=one_hot(y_val)
print(y_train.shape)

(45000,)
(10, 45000)


In [7]:
def get_batch(X_train,y_train,batch_size=100):
    ix = np.random.choice(X_train.shape[1], batch_size, replace=True)
    X_train_batch=X_train[:,ix]
    y_train_batch=y_train[:,ix]
    return X_train_batch.T,y_train_batch.T

In [8]:
X_train.shape

(3072, 45000)

In [9]:
def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=name)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [10]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [11]:
#first conv layer


CONV_1_DEPTH = 64
CONV_1_SIZE = 5
CONV_2_DEPTH = 64
CONV_2_SIZE = 5
CONV_3_DEPTH = 64
CONV_3_SIZE = 3
CONV_4_DEPTH = 64
CONV_4_SIZE = 3
epsilon = 1e-3
FULLY_CONNECTED_SIZE = 1024

x = tf.placeholder(tf.float32, shape=[None, 3072], name="init_x")
y_ = tf.placeholder(tf.float32, shape=[None, 10], name="init_y")

W_conv1 = weight_variable([CONV_1_SIZE, CONV_1_SIZE, 3, CONV_1_DEPTH], name="conv_1_weights")
b_conv1 = bias_variable([CONV_1_DEPTH])
#x_image = tf.reshape(x, [-1, 28, 28, 1])


x_image = tf.reshape(x, [-1, CONV_1_DEPTH, CONV_1_DEPTH, 3])
conv_1 = conv2d(x_image, W_conv1) + b_conv1

batch_mean1, batch_var1 = tf.nn.moments(conv_1,[0])
scale1 = tf.Variable(tf.ones([64]))
beta1 = tf.Variable(tf.zeros([64]))
batch1 = tf.nn.batch_normalization(conv_1,batch_mean1,batch_var1,beta1,scale1,epsilon)

h_conv1 = tf.nn.relu(batch1)
h_pool1 = max_pool_2x2(h_conv1)

#second conv layer
W_conv2 = weight_variable([CONV_2_SIZE, CONV_2_SIZE, CONV_1_DEPTH, 64], name="conv_2_weights")
b_conv2 = bias_variable([64])
conv_2 = conv2d(h_pool1, W_conv2) + b_conv2

batch_mean2, batch_var2 = tf.nn.moments(conv_2,[0])
scale2 = tf.Variable(tf.ones([64]))
beta2 = tf.Variable(tf.zeros([64]))
batch2 = tf.nn.batch_normalization(conv_2,batch_mean2,batch_var2,beta2,scale2,epsilon)

h_conv2 = tf.nn.relu(batch2)
h_pool2 = max_pool_2x2(h_conv2)



#third conv layer
W_conv3 = weight_variable([CONV_3_SIZE, CONV_3_SIZE, CONV_3_DEPTH, 64], name="conv_3_weights")
b_conv3 = bias_variable([64])
conv_3 = conv2d(h_pool2, W_conv3) + b_conv3

batch_mean3, batch_var3 = tf.nn.moments(conv_3,[0])
scale3 = tf.Variable(tf.ones([64]))
beta3 = tf.Variable(tf.zeros([64]))
batch3 = tf.nn.batch_normalization(conv_3,batch_mean3,batch_var3,beta3,scale3,epsilon)

h_conv3 = tf.nn.relu(batch3)
h_pool3 = max_pool_2x2(h_conv3)


#fourth conv layer
W_conv4 = weight_variable([CONV_4_SIZE, CONV_4_SIZE, CONV_4_DEPTH, 64], name="conv_4_weights")
b_conv4 = bias_variable([64])
conv_4 = conv2d(h_pool3, W_conv4) + b_conv4

batch_mean4, batch_var4 = tf.nn.moments(conv_4,[0])
scale4 = tf.Variable(tf.ones([64]))
beta4 = tf.Variable(tf.zeros([64]))
batch4 = tf.nn.batch_normalization(conv_4,batch_mean4,batch_var4,beta4,scale4,epsilon)

h_conv4 = tf.nn.relu(batch4)
h_pool4 = max_pool_2x2(h_conv4)


#dense layer
W_fc1 = weight_variable([2 * 2 * 64, FULLY_CONNECTED_SIZE], name="first_fc")
b_fc1 = bias_variable([FULLY_CONNECTED_SIZE])

h_pool4_flat = tf.reshape(h_pool4, [-1, 2*2*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool4_flat, W_fc1) + b_fc1)


#readout
W_fc2 = weight_variable([FULLY_CONNECTED_SIZE, 10], name="last_fc")
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

beta = 0.01
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# l2 regularization
regularizer = tf.nn.l2_loss(W_fc2) + tf.nn.l2_loss(W_fc1) + tf.nn.l2_loss(W_conv1) + tf.nn.l2_loss(W_conv2) + tf.nn.l2_loss(W_conv3) + tf.nn.l2_loss(W_conv4)
loss = tf.reduce_mean(cross_entropy + beta * regularizer)
#tf.summary.scalar("loss", loss)
#train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(.5).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#tf.summary.scalar("accuracy", accuracy)

# merge all summaries into a single "operation" which we can execute in a session 
#summary_op = tf.merge_all_summaries()

with tf.Session() as sess:
    #sess = tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.run(tf.global_variables_initializer())
    #writer = tf.summary.FileWriter("output", sess.graph)
    for i in range(10000):
    #    batch = mnist.train.next_batch(50)
        batch_x,batch_y = get_batch(X_train,y_train,batch_size=100)
        #print("batch x shape", batch_x.shape, batch_x.dtype)
        train_step.run(feed_dict={x: batch_x, y_: batch_y})
        
        # write log
        #writer.add_summary(summary, i)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch_x, y_: batch_y})
            print('step %d, training accuracy %g' % (i, train_accuracy))
            print('test accuracy %g' % accuracy.eval(feed_dict={x: X_val.T, y_: y_val.T}))
    
    # network trained, now make predictions on the test set
    prediction = tf.argmax(y_conv,1)
    preds = prediction.eval(feed_dict={x: X_test.T})
    np.save("hw2_ans1_" + UNI + ".npy", preds)
    

step 0, training accuracy 0.13
test accuracy 0.1058
step 100, training accuracy 0.29
test accuracy 0.1646
step 200, training accuracy 0.28
test accuracy 0.2252
step 300, training accuracy 0.24
test accuracy 0.24
step 400, training accuracy 0.35
test accuracy 0.256
step 500, training accuracy 0.4
test accuracy 0.2738
step 600, training accuracy 0.37
test accuracy 0.2876
step 700, training accuracy 0.3
test accuracy 0.2884
step 800, training accuracy 0.39
test accuracy 0.3032
step 900, training accuracy 0.41
test accuracy 0.3024
step 1000, training accuracy 0.39
test accuracy 0.3074
step 1100, training accuracy 0.33
test accuracy 0.3132
step 1200, training accuracy 0.36
test accuracy 0.3322
step 1300, training accuracy 0.41
test accuracy 0.3258
step 1400, training accuracy 0.49
test accuracy 0.3318
step 1500, training accuracy 0.32
test accuracy 0.331
step 1600, training accuracy 0.43
test accuracy 0.3264
step 1700, training accuracy 0.44
test accuracy 0.3516
step 1800, training accuracy

NameError: name 'writer' is not defined

In [ ]:
preds